In [1]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 9.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=2f7d80194a486982294735cd3280282792acb63c6a4e6d2e93057a295aa1f5e4
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=e2eaf88755147183a37f3079521c8d098cfb1d2208c22017f994f65413028481
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [3]:
tf.__version__

'2.4.1'

In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [5]:
(train_images, train_labels), (test_images,test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
fashion_mnist

<module 'tensorflow.keras.datasets.fashion_mnist' from '/usr/local/lib/python3.7/dist-packages/tensorflow/keras/datasets/fashion_mnist/__init__.py'>

In [7]:
train_images

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [8]:
train_labels

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

In [9]:
train_images = train_images / 255.0

In [10]:
test_images = test_images / 255.0

In [11]:
train_images[0].shape

(28, 28)

In [12]:
train_images = train_images.reshape(len(train_images),28,28,1)

In [13]:
test_images = test_images.reshape(len(test_images),28,28,1)

In [14]:
def build_model(hp):
  model = keras.Sequential([
                            keras.layers.Conv2D(filters=hp.Int('conv_1_filter',min_value=32, max_value=128, step=16), kernel_size=hp.Choice('conv_1_kernel', values=[3,5]),activation='relu',input_shape=(28,28,1)), 
                            keras.layers.Conv2D(filters=hp.Int('conv_2_filter',min_value=32, max_value=64, step=16), kernel_size=hp.Choice('conv_2_kernel', values=[3,5]), activation='relu'),
                            keras.layers.Flatten(),
                            keras.layers.Dense(units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16), activation='relu'),
                            keras.layers.Dense(10,activation='softmax')
                            ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values = [1e-2,1e-3])),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy']
                )
  return model

In [15]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [16]:
tuner_search = RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output', project_name='Fashion MNist')

In [17]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 18s]
val_accuracy: 0.9108333587646484

Best val_accuracy So Far: 0.9108333587646484
Total elapsed time: 00h 01m 44s
INFO:tensorflow:Oracle triggered exit


In [18]:
model = tuner_search.get_best_models(num_models=1)[0]

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        32288     
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                589856    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 623,594
Trainable params: 623,594
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1625 - accuracy: 0.9419 - val_loss: 0.2706 - val_accuracy: 0.9065
Epoch 5/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.1276 - accuracy: 0.9537 - val_loss: 0.2718 - val_accuracy: 0.9162
Epoch 6/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0975 - accuracy: 0.9650 - val_loss: 0.3007 - val_accuracy: 0.9092
Epoch 7/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0752 - accuracy: 0.9719 - val_loss: 0.3406 - val_accuracy: 0.9033
Epoch 8/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0533 - accuracy: 0.9806 - val_loss: 0.3893 - val_accuracy: 0.9113
Epoch 9/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0494 - accuracy: 0.9822 - val_loss: 0.4261 - val_accuracy: 0.9065
Epoch 10/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0393 - accuracy: 0.9860 - val_loss: 0.4896 - val_accuracy